In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from torch.utils.data import TensorDataset, DataLoader

In [3]:
# config
epochs = 60
learning_rate = 0.005

in_features=1152
h1 = 1000
h2 = 800
h3 = 500
out_features = 64

batch_size = 512

project = "chess nn1 linear model 3, my moves, 1 label"
architecture = "Linear network with 2 hidden layers, my moves, 1 label",
dataset = "max rapid games",

In [4]:
import wandb

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project = project,
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": learning_rate,
    "architecture": architecture,
    "dataset": dataset,
    "epochs": epochs,
    "h1": h1,
    "h2": h2
    }
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: python-wizard. Use `wandb login --relogin` to force relogin


In [5]:
cuda_av = torch.cuda.is_available()
cuda_av


True

In [6]:
if cuda_av:
    cuda_id = torch.cuda.current_device()
    print(f'CUDA device ID: {cuda_id}')
    print(f"CUDA device name: {torch.cuda.get_device_name(cuda_id)}")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

CUDA device ID: 0
CUDA device name: NVIDIA GeForce GTX 1070


'cuda'

In [7]:
data_training = torch.load('tensor_data/nn1_rapid_games_data_training_09_02_24.pt')
print(data_training.shape)
data_training.to(device)

torch.Size([134152, 1152])


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float64)

In [8]:
data_testing = torch.load('tensor_data/nn1_rapid_games_data_testing_09_02_24.pt')
print(data_testing.shape)
data_testing.to(device)

torch.Size([33538, 1152])


tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0', dtype=torch.float64)

In [9]:
lables_training = torch.load('tensor_data/nn1_rapid_games_lables_training_09_02_24.pt')
print(lables_training.shape)
lables_training.to(device)

torch.Size([134152])


tensor([14, 58, 24,  ...,  3, 50, 21], device='cuda:0')

In [10]:
lables_testing = torch.load('tensor_data/nn1_rapid_games_lables_testing_09_02_24.pt')
print(lables_testing.shape)
lables_testing.to(device)

torch.Size([33538])


tensor([56,  9, 11,  ..., 34, 26, 12], device='cuda:0')

In [ ]:
torch.manual_seed(42)

In [11]:
training_dataset = TensorDataset(data_training, lables_training)
testing_dataset = TensorDataset(data_testing, lables_testing)

In [12]:
data_loader_training = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
data_loader_testing = DataLoader(testing_dataset, batch_size=batch_size)


In [13]:
torch.set_default_dtype(torch.float64)

In [14]:
class Model(nn.Module):

    def __init__(self, in_features=in_features, h1=h1, h2=h2, h3=h3, out_features=64): #out = 64 squares to move to 

        super().__init__()

        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        # self.fc2 = nn.Linear(h1, h2)
        # self.fc3 = nn.Linear(h2, h3)
        # self.fc3 = nn.Linear(h2, h3)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # x = F.relu(self.fc3(x))
        x = self.out(x)
        
        return x



In [15]:
model = Model()

In [16]:
# wandb.watch(model)

In [17]:
print(model.parameters())

<generator object Module.parameters at 0x7f70709050e0>


In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
wandb.watch(model, criterion, log="all", log_freq=10)

[]

In [20]:
import time
start_time = time.time()

for e in range(epochs):

    train_correct = 0
    test_correct = 0

    for X_train, y_train in data_loader_training:
        
        y_pred = model.forward(X_train)
        loss = criterion(y_pred, y_train)

        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_correct = (predicted == y_train).sum()
        train_correct += batch_correct


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if e % 5 == 0 or e < 10:
        for X_test, y_test in data_loader_testing:

            with torch.no_grad():
                y_pred_testing = model(X_test)
                loss_testing = criterion(y_pred_testing, y_test)

                predicted = torch.max(y_pred.data, 1)[1]
                batch_correct = (predicted == y_train).sum()
                test_correct += batch_correct

    # wandb.log({"epoch": e, "loss": loss, "loss testing": loss_testing, 'accuracy_like': intersections.mean(), 'accuracy_like_testing': intersections_testing.mean()})
    wandb.log({"loss": loss, "loss testing": loss_testing, 'accuracy': train_correct/data_training.shape[0], 'accuracy testing': test_correct/data_testing.shape[0]})


print(f'\nDuration: {time.time() - start_time:.0f} seconds') 


Duration: 1029 seconds


In [21]:
wandb.finish()

accuracy,▁▁▂▂▃▃▄▄▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇████████████
accuracy testing,▂▂▃▃▃▄▄▅▁▁▆▁▁▁▁▁▁▇▁▁▇▁▁▁▁▁▁█▁▁█▁▁▁▁▁▁█▁▁
loss,██▇▇▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
loss testing,█▇▆▅▄▃▃▂▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅
accuracy,0.75821
accuracy testing,0.0
loss,0.74686
loss testing,3.04701


In [22]:
# torch.save(model, 'models/nn1_model_linear_800_800.pt')

In [25]:
torch.save(model.state_dict(), 'models/nn1_model_linear_1000_800.pt')